In [50]:
# imports
import sys, ast
# Need to do this so we can import the modules in `../..libs`
sys.path.insert(0, '../')
import os, tqdm, json, re
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

# custom libraries
import libs.analysis as analysis
import libs.utils as utils
import libs.get_data as get_data

# misc visualisation if needed
import matplotlib.pyplot as plt
import seaborn as sns

Declaring test/analysis start and end dates as both a string and an np.datetime64 type. 

In [51]:
data_dir = f"./data/"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

In [52]:
START = "2023-11-20"
END = "2023-12-05"

In [53]:
START_DATE = np.datetime64(START)
END_DATE = np.datetime64(END)

Get the data from BigQuery, could be adapted to other sources.
This function is supposed to create or update a CSV file in the `./data` dir that matches the SQL file name.
I think the update just appends and creates duplicate rows, safer to just delete the `./data` dir and call query again.
By default, this gets all data from the specified start date until "yesterday". This can be tweaked directly in the `get_data.update_local_data` func

In [54]:
# This is commented out because the query is dummy and will do nothing.
# get_data.update_local_data("SQL-Query", TEST_start_date=START)

Open the data file and apply any basic processing/filtering/transforming/dark-magic

`utils.status(DataFrame)` will just print the dataframe shape and `DataFrame.head()`, saves some typing

In [55]:
# For example purposes, this is commented out and replaced with dummy data.
# raw_events_data = utils.load_bq_df("./data/SQL-Query.csv")
raw_events_data = pd.DataFrame({
    "event_date": np.random.choice(pd.date_range(START_DATE, END_DATE), 12),
    "optimisation_id": ["test01"] * 12,
    "optimisation_variant": ["Control", "Variation 1"] * 6,
    "device_category": ["desktop", "tablet", "mobile"] * 4,
    "impressions": np.random.randint(100, 1000, 12),
    "purchases": np.random.randint(0, 50, 12)
})

print(f"Start Date: {raw_events_data.event_date.min()}")
print(f"End Date: {raw_events_data.event_date.max()}")
utils.status(raw_events_data)

Start Date: 2023-11-20 00:00:00
End Date: 2023-12-05 00:00:00
Shape: (12, 6)


,event_date,optimisation_id,optimisation_variant,device_category,impressions,purchases
0,2023-12-01,test01,Control,desktop,105,28
1,2023-11-29,test01,Variation 1,tablet,715,10
2,2023-12-02,test01,Control,mobile,952,17
3,2023-11-20,test01,Variation 1,desktop,996,49
4,2023-11-28,test01,Control,tablet,754,2


You can also create a device report that'll provide an overview of the device distribution. This can be used to dynamically create a slide in the report.

In [56]:
device_report = utils.create_device_report(raw_events_data, start=START, end=END)
device_report.to_csv("./data/device_report.csv")
device_report

,device_category,impressions,% of total,start_date,end_date
0,desktop,2256,27.421903,2023-11-20,2023-12-05
1,mobile,2869,34.872979,2023-11-20,2023-12-05
2,tablet,3102,37.705117,2023-11-20,2023-12-05


Grouping and aggregations

In [57]:
data = raw_events_data.groupby(["optimisation_id", "optimisation_variant", "device_category"]).sum(numeric_only=True).reset_index()
utils.status(data, n=6)

Shape: (6, 5)


,optimisation_id,optimisation_variant,device_category,impressions,purchases
0,test01,Control,desktop,384,76
1,test01,Control,mobile,1143,54
2,test01,Control,tablet,1541,9
3,test01,Variation 1,desktop,1872,52
4,test01,Variation 1,mobile,1726,58
5,test01,Variation 1,tablet,1561,12


Split users out into segments for analysis

In [58]:
segment_output_dir = "./data/segmented"
if not os.path.exists(segment_output_dir):
    os.makedirs(segment_output_dir)

In [59]:
core_dims = ["optimisation_id", "device_category"]

In [60]:
all_users = data.drop(core_dims, axis=1)
all_users["segment"] = "All Users"
all_users = all_users.groupby(["segment", "optimisation_variant"], as_index=False).agg("sum")
all_users.to_csv(f"{segment_output_dir}/all_users.csv")
utils.status(all_users)

Shape: (2, 4)


,segment,optimisation_variant,impressions,purchases
0,All Users,Control,3068,139
1,All Users,Variation 1,5159,122


In [61]:
desktop_tablet_users = data[data.device_category != "mobile"].drop(core_dims, axis=1)
desktop_tablet_users["segment"] = "Desktop & Tablet Users"
desktop_tablet_users = desktop_tablet_users.groupby(["segment", "optimisation_variant"], as_index=False).agg("sum")
desktop_tablet_users.to_csv(f"{segment_output_dir}/desktop_tablet_users.csv")
utils.status(desktop_tablet_users)

Shape: (2, 4)


,segment,optimisation_variant,impressions,purchases
0,Desktop & Tablet Users,Control,1925,85
1,Desktop & Tablet Users,Variation 1,3433,64


In [62]:
mobile_users = data[data.device_category == "mobile"].drop(core_dims, axis=1)
mobile_users["segment"] = "Mobile Users"
mobile_users = mobile_users.groupby(["segment", "optimisation_variant"], as_index=False).agg("sum")
mobile_users.to_csv(f"{segment_output_dir}/mobile_users.csv")
utils.status(mobile_users)

Shape: (2, 4)


,segment,optimisation_variant,impressions,purchases
0,Mobile Users,Control,1143,54
1,Mobile Users,Variation 1,1726,58


Now, we move over to `./analysis.ipynb`